moves to FEN


In [1]:
moves = ["'e4'", " 'c6'", " 'Nf3'", " 'd5'", " 'e5'", " 'e6'", " 'd4'", " 'Ne7'", " 'Bd3'", " 'c5'", " 'dxc5'",
 " 'Nd7'", " 'Be3'", " 'Qc7'", " 'b4'", " 'Nxe5'"," 'Nxe5'"," 'Qxe5'"," 'Bb5+'", " 'Bd7'"," 'Bxd7+'", " 'Kxd7'",
 " 'Qd4'", " 'Ng6'", " 'O-O'", " 'Qxd4'", " 'Bxd4'", " 'e5'", " 'Bb2'", " 'a5'", " 'b5'", " 'Bxc5'", " 'Nd2'",
 " 'b6'", " 'c4'", " 'd4'"," 'Ne4'", " 'Nf4'", " 'Nxc5+'", " 'bxc5'", " 'Ba3'", " 'Kd6'", " 'Rab1'"," 'Rhc8'",
 " 'g3'", " 'Nd3'", " 'Rfd1'", " 'Nb4'", " 'f4'", " 'e4'", " 'g4'", " 'e3'", " 'f5'", " 'Nxa2'", " 'Rf1'",
 " 'Nc3'", " 'Rbc1'", " 'e2'", " 'Rf4'", " 'Re8'", " 'Rxd4+'", " 'Ke5'", " 'Rd2'", " 'Kf6'", " 'Bb2'", " 'e1=Q+'",
 " 'Rxe1'", " 'Rxe1+'", " 'Kf2'", " 'Rae8'", " 'Bxc3+'", " 'Kg5'", " 'Bxa5'", " 'Rh1'", " 'Kg3'", " 'Re3+'", " 'Kg2'",
 " 'Rd1'", " 'Rxd1'"]

In [29]:
import numpy as np
# w e pos, b e neg. p = 1, r=1, n=2, b=3, q=4, k=5
x = np.zeros((8,8)) # empty game board

fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR" # start pos
fen = fen.split('/')        # line by line
pieces_list = [*"prnbqk"]   # pieces
for idx, line in enumerate(fen):
    if any(elem.lower() in pieces_list for elem in [*line]):    
        pieces = [pieces_list.index(piece.lower())+1 for piece in [*line]]
        pieces = [piece * -1 if line.islower() else piece for piece in pieces] #makin black
        x[idx] = np.array(pieces)

x

array([[-2., -3., -4., -5., -6., -4., -3., -2.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 2.,  3.,  4.,  5.,  6.,  4.,  3.,  2.]])

In [ ]:
piece_int_map = dict
pieces_list = [*"prnbqk"]   # pieces
for piece in pieces_list:
    piece_int_map
    


In [43]:
moves = ["'e4'", " 'c6'", " 'Nf3'"] # vil ha d om til tall, x ,y
char_int_map = [*'abcdefgh']
move = [*moves[0].strip("'")]
move = [char_int_map.index(mv) if mv in char_int_map else int(mv) for mv in move]
x[move[0]][move[1]]



0.0

In [ ]:
def moves_to_fen(moves: list) -> string: #ex: 'r1b1k1nr/p2p1pNp/n2B4/1p1NP2P/6P1/3P1Q2/P1P1K3/q5b1'
    fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR" # start pos
    # enten update fen for hvert move, eller finn ut ka slutt posisjon e
    # kan lag en matrix
    board_mat



    return fen


In [ ]:
moves_to_fen(moves)